# **Santander EDA and Prediction**

Conjunto de datos utilizado: Predicción de transacciones de clientes de Santander

Introducción
En este desafío, Santander invita a Kagglers a ayudarlos a identificar qué clientes realizarán una transacción específica en el futuro, independientemente de la cantidad de dinero que se realice. Los datos aportados para esta competición tienen la misma estructura que los datos reales de los que disponen para solucionar este problema.
Los datos se anonimizan, cada fila contiene 200 valores numéricos identificados solo con un número.
A continuación, exploraremos los datos, los prepararemos para un modelo, entrenaremos un modelo y predeciremos el valor objetivo para el conjunto de prueba, luego prepararemos un envío.



# **Prepárese para el análisis de datos**

Cargar paquetes


In [ ]:
import gc
import os
import logging
import datetime
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import lightgbm as lgb
from tqdm import tqdm_notebook
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import StratifiedKFold
warnings.filterwarnings('ignore')

# **Cargar datos**

Veamos qué archivos de datos están disponibles.

In [ ]:
IS_LOCAL = False
if(IS_LOCAL):
    PATH="../input/Santander/"
else:
    PATH="../input/"
os.listdir(PATH)

Carguemos el tren y probemos los archivos de datos.

In [ ]:
%%time
train_df = pd.read_csv(PATH+"train.csv")
test_df = pd.read_csv(PATH+"test.csv")

# **Exploración de datos**
Comprobar los datos
Revisemos el tren y el equipo de prueba.

In [ ]:
train_df.shape, test_df.shape

Tanto los datos de tren como los de prueba tienen 200.000 entradas y 202, respectivamente 201 columnas.
Echemos un vistazo al conjunto de datos de entrenamiento y prueba.

In [ ]:
train_df.head()

In [ ]:
test_df.head()

El tren contiene:

ID_code (cadena);
objetivo;
200 variables numéricas, nombradas de var_0 a var_199;
La prueba contiene:

ID_code (cadena);
200 variables numéricas, nombradas de var_0 a var_199;
Comprobemos si faltan datos. También comprobaremos el tipo de datos.

Comprobamos el primer tren.

In [ ]:
def missing_data(data):
    total = data.isnull().sum()
    percent = (data.isnull().sum()/data.isnull().count()*100)
    tt = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
    types = []
    for col in data.columns:
        dtype = str(data[col].dtype)
        types.append(dtype)
    tt['Types'] = types
    return(np.transpose(tt))

In [ ]:
%%time
missing_data(train_df)

Aquí verificamos el conjunto de datos de prueba.

In [ ]:
%%time
missing_data(test_df)

No faltan datos en los conjuntos de datos de prueba y tren. Comprobemos los valores numéricos en el conjunto de datos de prueba y tren.

In [ ]:
%%time
train_df.describe()

In [ ]:
%time
test_df.describe()


Podemos hacer algunas observaciones aquí:

* la desviación estándar es relativamente grande para los datos de variables de prueba y de tren;
* Los valores min, max, mean, sdt para los datos de prueba y de tren parecen bastante cercanos;
* los valores medios se distribuyen en un amplio rango.
El número de valores en el tren y en el conjunto de prueba es el mismo. Tracemos el diagrama de dispersión para el tren y el conjunto de prueba para algunas de las características.


In [ ]:
def plot_feature_scatter(df1, df2, features):
    i = 0
    sns.set_style('whitegrid')
    plt.figure()
    fig, ax = plt.subplots(4,4,figsize=(14,14))

    for feature in features:
        i += 1
        plt.subplot(4,4,i)
        plt.scatter(df1[feature], df2[feature], marker='+')
        plt.xlabel(feature, fontsize=9)
    plt.show();

Mostraremos solo el 5% de los datos. En el eje x mostramos los valores del tren y en el eje y mostramos los valores de prueba.

In [ ]:
features = ['var_0', 'var_1','var_2','var_3', 'var_4', 'var_5', 'var_6', 'var_7', 
           'var_8', 'var_9', 'var_10','var_11','var_12', 'var_13', 'var_14', 'var_15', 
           ]
plot_feature_scatter(train_df[::20],test_df[::20], features)

Comprobemos la distribución del valor objetivo en el conjunto de datos del tren.

In [ ]:
sns.countplot(train_df['target'], palette='Set3')

In [ ]:
print("There are {}% target values with 1".format(100 * train_df["target"].value_counts()[1]/train_df.shape[0]))

Los datos están desequilibrados con respecto al valor objetivo. 

Gráficos de densidad de características

Vamos a mostrar ahora la gráfica de densidad de las variables en el conjunto de datos del tren.

Representamos con diferentes colores la distribución para valores con valor objetivo 0 y 1.

In [ ]:
def plot_feature_distribution(df1, df2, label1, label2, features):
    i = 0
    sns.set_style('whitegrid')
    plt.figure()
    fig, ax = plt.subplots(10,10,figsize=(18,22))

    for feature in features:
        i += 1
        plt.subplot(10,10,i)
        sns.distplot(df1[feature], hist=False,label=label1)
        sns.distplot(df2[feature], hist=False,label=label2)
        plt.xlabel(feature, fontsize=9)
        locs, labels = plt.xticks()
        plt.tick_params(axis='x', which='major', labelsize=6, pad=-6)
        plt.tick_params(axis='y', which='major', labelsize=6)
    plt.show();

Los primeros 100 valores se muestran en la siguiente celda. Presione Salida para mostrar los gráficos.

In [ ]:
t0 = train_df.loc[train_df['target'] == 0]
t1 = train_df.loc[train_df['target'] == 1]
features = train_df.columns.values[2:102]
plot_feature_distribution(t0, t1, '0', '1', features)

Los siguientes 100 valores se muestran en la siguiente celda. Presione Salida para mostrar los gráficos.

In [ ]:
features = train_df.columns.values[102:202]
plot_feature_distribution(t0, t1, '0', '1', features)

Podemos observar que hay un número considerable de características con una distribución significativa diferente para los dos valores objetivo.
Por ejemplo, var_0, var_1, var_2, var_5, var_9, var_13, var_106, var_109, var_139 y muchos otros.

Además, algunas características, como var_2, var_13, var_26, var_55, var_175, var_184, var_196 muestran una distribución que se asemeja a una distribución bivariada.

Tendremos esto en cuenta en el futuro para la selección de las características de nuestro modelo de predicción.

Veamos ahora la distribución de las mismas características en paralelo en conjuntos de datos de prueba y entrenamiento.

Los primeros 100 valores se muestran en la siguiente celda. Presione Salida para mostrar los gráficos.

In [ ]:
features = train_df.columns.values[2:102]
plot_feature_distribution(train_df, test_df, 'train', 'test', features)

Los siguientes 100 valores se muestran en la siguiente celda. Presione Salida para mostrar los gráficos.

In [ ]:
features = train_df.columns.values[102:202]
plot_feature_distribution(train_df, test_df, 'train', 'test', features)

El tren y la prueba parecen estar bien equilibrados con respecto a la distribución de las variables numéricas.

Distribución de media y std

Comprobemos la distribución de los valores medios por fila en el tren y el conjunto de prueba.

In [ ]:
plt.figure(figsize=(16,6))
features = train_df.columns.values[2:202]
plt.title("Distribution of mean values per row in the train and test set")
sns.distplot(train_df[features].mean(axis=1),color="green", kde=True,bins=120, label='train')
sns.distplot(test_df[features].mean(axis=1),color="blue", kde=True,bins=120, label='test')
plt.legend()
plt.show()

Comprobemos la distribución de los valores medios por columnas en el tren y el conjunto de prueba.

In [ ]:
plt.figure(figsize=(16,6))
plt.title("Distribution of mean values per column in the train and test set")
sns.distplot(train_df[features].mean(axis=0),color="magenta",kde=True,bins=120, label='train')
sns.distplot(test_df[features].mean(axis=0),color="darkblue", kde=True,bins=120, label='test')
plt.legend()
plt.show()

Vamos a mostrar la distribución de la desviación estándar de los valores por fila para los conjuntos de datos de prueba y entrenamiento.

In [ ]:
plt.figure(figsize=(16,6))
plt.title("Distribution of std values per row in the train and test set")
sns.distplot(train_df[features].std(axis=1),color="black", kde=True,bins=120, label='train')
sns.distplot(test_df[features].std(axis=1),color="red", kde=True,bins=120, label='test')
plt.legend();plt.show()

Comprobemos la distribución de la desviación estándar de valores por columnas en los conjuntos de datos de prueba y tren.

In [ ]:
plt.figure(figsize=(16,6))
plt.title("Distribution of std values per column in the train and test set")
sns.distplot(train_df[features].std(axis=0),color="blue",kde=True,bins=120, label='train')
sns.distplot(test_df[features].std(axis=0),color="green", kde=True,bins=120, label='test')
plt.legend(); plt.show()

Veamos ahora la distribución del valor medio por fila en el conjunto de datos del tren, agrupados por valor del objetivo.

In [ ]:
t0 = train_df.loc[train_df['target'] == 0]
t1 = train_df.loc[train_df['target'] == 1]
plt.figure(figsize=(16,6))
plt.title("Distribution of mean values per row in the train set")
sns.distplot(t0[features].mean(axis=1),color="red", kde=True,bins=120, label='target = 0')
sns.distplot(t1[features].mean(axis=1),color="blue", kde=True,bins=120, label='target = 1')
plt.legend(); plt.show()

Veamos ahora la distribución del valor medio por columna en el conjunto de datos del tren, agrupados por valor del objetivo.

In [ ]:
plt.figure(figsize=(16,6))
plt.title("Distribution of mean values per column in the train set")
sns.distplot(t0[features].mean(axis=0),color="green", kde=True,bins=120, label='target = 0')
sns.distplot(t1[features].mean(axis=0),color="darkblue", kde=True,bins=120, label='target = 1')
plt.legend(); plt.show()

# Distribución de min y max

Comprobemos la distribución de min por fila en el tren y el conjunto de prueba.

In [ ]:
plt.figure(figsize=(16,6))
features = train_df.columns.values[2:202]
plt.title("Distribution of min values per row in the train and test set")
sns.distplot(train_df[features].min(axis=1),color="red", kde=True,bins=120, label='train')
sns.distplot(test_df[features].min(axis=1),color="orange", kde=True,bins=120, label='test')
plt.legend()
plt.show()

Se observa una larga cola para los valores más bajos para ambos, extendida hasta -80 para el equipo de prueba.

Vamos a mostrar ahora la distribución de min por columna en el tren y el conjunto de prueba.

In [ ]:
plt.figure(figsize=(16,6))
features = train_df.columns.values[2:202]
plt.title("Distribution of min values per column in the train and test set")
sns.distplot(train_df[features].min(axis=0),color="magenta", kde=True,bins=120, label='train')
sns.distplot(test_df[features].min(axis=0),color="darkblue", kde=True,bins=120, label='test')
plt.legend()
plt.show()

Veamos ahora la distribución de los valores máximos por filas para el tren y el conjunto de prueba.

In [ ]:
plt.figure(figsize=(16,6))
features = train_df.columns.values[2:202]
plt.title("Distribution of max values per row in the train and test set")
sns.distplot(train_df[features].max(axis=1),color="brown", kde=True,bins=120, label='train')
sns.distplot(test_df[features].max(axis=1),color="yellow", kde=True,bins=120, label='test')
plt.legend()
plt.show()

Vamos a mostrar ahora la distribución máxima en columnas para el tren y el conjunto de prueba.

In [ ]:
plt.figure(figsize=(16,6))
features = train_df.columns.values[2:202]
plt.title("Distribution of max values per column in the train and test set")
sns.distplot(train_df[features].max(axis=0),color="blue", kde=True,bins=120, label='train')
sns.distplot(test_df[features].max(axis=0),color="red", kde=True,bins=120, label='test')
plt.legend()
plt.show()

Vamos a mostrar ahora las distribuciones de los valores mínimos por fila en el conjunto de trenes, separados por los valores del objetivo (0 y 1).

In [ ]:
t0 = train_df.loc[train_df['target'] == 0]
t1 = train_df.loc[train_df['target'] == 1]
plt.figure(figsize=(16,6))
plt.title("Distribution of min values per row in the train set")
sns.distplot(t0[features].min(axis=1),color="orange", kde=True,bins=120, label='target = 0')
sns.distplot(t1[features].min(axis=1),color="darkblue", kde=True,bins=120, label='target = 1')
plt.legend(); plt.show()

Mostramos aquí la distribución de valores mínimos por columnas en el conjunto de trenes.

In [ ]:
plt.figure(figsize=(16,6))
plt.title("Distribution of min values per column in the train set")
sns.distplot(t0[features].min(axis=0),color="red", kde=True,bins=120, label='target = 0')
sns.distplot(t1[features].min(axis=0),color="blue", kde=True,bins=120, label='target = 1')
plt.legend(); plt.show()

Vamos a mostrar ahora la distribución de los valores máximos por fila en el conjunto de trenes.

In [ ]:
plt.figure(figsize=(16,6))
plt.title("Distribution of max values per row in the train set")
sns.distplot(t0[features].max(axis=1),color="gold", kde=True,bins=120, label='target = 0')
sns.distplot(t1[features].max(axis=1),color="darkblue", kde=True,bins=120, label='target = 1')
plt.legend(); plt.show()

Vamos a mostrar también la distribución de los valores máximos por columnas en el conjunto de trenes.

In [ ]:
plt.figure(figsize=(16,6))
plt.title("Distribution of max values per column in the train set")
sns.distplot(t0[features].max(axis=0),color="red", kde=True,bins=120, label='target = 0')
sns.distplot(t1[features].max(axis=0),color="blue", kde=True,bins=120, label='target = 1')
plt.legend(); plt.show()

# Distribución de sesgo y curtosis

Veamos ahora cuál es la distribución de valores de sesgo por filas y columnas.

Veamos primero la distribución de la asimetría calculada por filas en los conjuntos de tren y prueba.

In [ ]:
plt.figure(figsize=(16,6))
plt.title("Distribution of skew per row in the train and test set")
sns.distplot(train_df[features].skew(axis=1),color="red", kde=True,bins=120, label='train')
sns.distplot(test_df[features].skew(axis=1),color="orange", kde=True,bins=120, label='test')
plt.legend()
plt.show()

Veamos primero la distribución de asimetría calculada por columnas en el tren y el conjunto de prueba.

In [ ]:
plt.figure(figsize=(16,6))
plt.title("Distribution of skew per column in the train and test set")
sns.distplot(train_df[features].skew(axis=0),color="magenta", kde=True,bins=120, label='train')
sns.distplot(test_df[features].skew(axis=0),color="darkblue", kde=True,bins=120, label='test')
plt.legend()
plt.show()

Veamos ahora cuál es la distribución de los valores de curtosis por filas y columnas.

Veamos primero la distribución de la curtosis calculada por filas en conjuntos de tren y prueba.

In [ ]:
plt.figure(figsize=(16,6))
plt.title("Distribution of kurtosis per row in the train and test set")
sns.distplot(train_df[features].kurtosis(axis=1),color="darkblue", kde=True,bins=120, label='train')
sns.distplot(test_df[features].kurtosis(axis=1),color="yellow", kde=True,bins=120, label='test')
plt.legend()
plt.show()

Veamos primero la distribución de la curtosis calculada por columnas en conjuntos de prueba y tren.

In [ ]:
plt.figure(figsize=(16,6))
plt.title("Distribution of kurtosis per column in the train and test set")
sns.distplot(train_df[features].kurtosis(axis=0),color="magenta", kde=True,bins=120, label='train')
sns.distplot(test_df[features].kurtosis(axis=0),color="green", kde=True,bins=120, label='test')
plt.legend()
plt.show()

Veamos ahora la distribución de la asimetría en filas en el tren separadas para los valores de destino 0 y 1.

In [ ]:
t0 = train_df.loc[train_df['target'] == 0]
t1 = train_df.loc[train_df['target'] == 1]
plt.figure(figsize=(16,6))
plt.title("Distribution of skew values per row in the train set")
sns.distplot(t0[features].skew(axis=1),color="red", kde=True,bins=120, label='target = 0')
sns.distplot(t1[features].skew(axis=1),color="blue", kde=True,bins=120, label='target = 1')
plt.legend(); plt.show()

Veamos ahora la distribución de la asimetría en las columnas en el tren separadas para los valores del objetivo 0 y 1.

In [ ]:
t0 = train_df.loc[train_df['target'] == 0]
t1 = train_df.loc[train_df['target'] == 1]
plt.figure(figsize=(16,6))
plt.title("Distribution of skew values per column in the train set")
sns.distplot(t0[features].skew(axis=0),color="red", kde=True,bins=120, label='target = 0')
sns.distplot(t1[features].skew(axis=0),color="blue", kde=True,bins=120, label='target = 1')
plt.legend(); plt.show()

Veamos ahora la distribución de la curtosis en filas en el tren separadas por valores de objetivo 0 y 1.

In [ ]:
t0 = train_df.loc[train_df['target'] == 0]
t1 = train_df.loc[train_df['target'] == 1]
plt.figure(figsize=(16,6))
plt.title("Distribution of kurtosis values per row in the train set")
sns.distplot(t0[features].kurtosis(axis=1),color="red", kde=True,bins=120, label='target = 0')
sns.distplot(t1[features].kurtosis(axis=1),color="blue", kde=True,bins=120, label='target = 1')
plt.legend(); plt.show()

Veamos ahora la distribución de la curtosis en columnas en el tren separadas por valores de objetivo 0 y 1.

In [ ]:
t0 = train_df.loc[train_df['target'] == 0]
t1 = train_df.loc[train_df['target'] == 1]
plt.figure(figsize=(16,6))
plt.title("Distribution of kurtosis values per column in the train set")
sns.distplot(t0[features].kurtosis(axis=0),color="red", kde=True,bins=120, label='target = 0')
sns.distplot(t1[features].kurtosis(axis=0),color="blue", kde=True,bins=120, label='target = 1')
plt.legend(); plt.show()

# Correlación de características

Calculamos ahora las correlaciones entre las características en el conjunto de trenes.

La siguiente tabla muestra las primeras 10 características menos correlacionadas.

In [ ]:
%%time
correlations = train_df[features].corr().abs().unstack().sort_values(kind="quicksort").reset_index()
correlations = correlations[correlations['level_0'] != correlations['level_1']]
correlations.head(10)

Veamos las características principales más correlacionadas, además de los mismos pares de características.

In [ ]:
correlations.tail(10)

Veamos también las características menos correlacionadas.

In [ ]:
correlations.head(10)

La correlación entre las características es muy pequeña.

# Valores duplicados

Veamos ahora cuántos valores duplicados existen por columna.

In [ ]:
%%time
features = train_df.columns.values[2:202]
unique_max_train = []
unique_max_test = []
for feature in features:
    values = train_df[feature].value_counts()
    unique_max_train.append([feature, values.max(), values.idxmax()])
    values = test_df[feature].value_counts()
    unique_max_test.append([feature, values.max(), values.idxmax()])

Vamos a mostrar el máximo de 15 valores duplicados por conjunto de trenes.

In [ ]:
np.transpose((pd.DataFrame(unique_max_train, columns=['Feature', 'Max duplicates', 'Value'])).\
            sort_values(by = 'Max duplicates', ascending=False).head(15))

Veamos también los 15 primeros valores duplicados por conjunto de prueba.

In [ ]:
np.transpose((pd.DataFrame(unique_max_test, columns=['Feature', 'Max duplicates', 'Value'])).\
            sort_values(by = 'Max duplicates', ascending=False).head(15))

Las mismas columnas en el tren y en el conjunto de prueba tienen el mismo número o muy cerca de duplicados de valores iguales o muy cercanos. Este es un patrón interesante que podríamos utilizar en el futuro.

# Ingeniería de características

Esta sección está bajo construcción.

Calculemos para comenzar algunos valores agregados para las características existentes.

In [ ]:
%%time
idx = features = train_df.columns.values[2:202]
for df in [test_df, train_df]:
    df['sum'] = df[idx].sum(axis=1)  
    df['min'] = df[idx].min(axis=1)
    df['max'] = df[idx].max(axis=1)
    df['mean'] = df[idx].mean(axis=1)
    df['std'] = df[idx].std(axis=1)
    df['skew'] = df[idx].skew(axis=1)
    df['kurt'] = df[idx].kurtosis(axis=1)
    df['med'] = df[idx].median(axis=1)

Revisemos las nuevas funciones creadas.

In [ ]:
train_df[train_df.columns[202:]].head()

In [ ]:
test_df[test_df.columns[201:]].head()

In [ ]:
def plot_new_feature_distribution(df1, df2, label1, label2, features):
    i = 0
    sns.set_style('whitegrid')
    plt.figure()
    fig, ax = plt.subplots(2,4,figsize=(18,8))

    for feature in features:
        i += 1
        plt.subplot(2,4,i)
        sns.kdeplot(df1[feature], bw=0.5,label=label1)
        sns.kdeplot(df2[feature], bw=0.5,label=label2)
        plt.xlabel(feature, fontsize=11)
        locs, labels = plt.xticks()
        plt.tick_params(axis='x', which='major', labelsize=8)
        plt.tick_params(axis='y', which='major', labelsize=8)
    plt.show();

Revisemos la distribución de estas nuevas funciones diseñadas.

Primero trazamos la distribución de nuevas características, agrupadas por el valor de los valores de "objetivo" correspondientes.

In [ ]:
t0 = train_df.loc[train_df['target'] == 0]
t1 = train_df.loc[train_df['target'] == 1]
features = train_df.columns.values[202:]
plot_new_feature_distribution(t0, t1, 'target: 0', 'target: 1', features)

Vamos a mostrar la distribución de los valores de las nuevas funciones para entrenar y probar.

In [ ]:
features = train_df.columns.values[202:]
plot_new_feature_distribution(train_df, test_df, 'train', 'test', features)

We add rounded features.  
**Note**: this is a work in progress, some of the features added here will be later dropped.

<code>
features = [c for c in train_df.columns if c not in ['ID_code', 'target']]
for feature in features:
    train_df['r2_'+feature] = np.round(train_df[feature], 2)
    test_df['r2_'+feature] = np.round(test_df[feature], 2)
    train_df['r1_'+feature] = np.round(train_df[feature], 1)
    test_df['r1_'+feature] = np.round(test_df[feature], 1)
</code>

Veamos cuántas funciones tenemos ahora.

In [ ]:
print('Train and test columns: {} {}'.format(len(train_df.columns), len(test_df.columns)))

# <a id='5'>Modelo</a>  


De la lista de columnas del tren, eliminamos el ID y el objetivo para formar la lista de características.

In [ ]:
features = [c for c in train_df.columns if c not in ['ID_code', 'target']]
target = train_df['target']

Definimos los hiperparámetros para el modelo.

In [ ]:
param = {
    'bagging_freq': 5,
    'bagging_fraction': 0.4,
    'boost_from_average':'false',
    'boost': 'gbdt',
    'feature_fraction': 0.05,
    'learning_rate': 0.01,
    'max_depth': -1,  
    'metric':'auc',
    'min_data_in_leaf': 80,
    'min_sum_hessian_in_leaf': 10.0,
    'num_leaves': 13,
    'num_threads': 8,
    'tree_learner': 'serial',
    'objective': 'binary', 
    'verbosity': 1
}

Ejecutamos el modelo.

In [ ]:
folds = StratifiedKFold(n_splits=10, shuffle=False, random_state=44000)
oof = np.zeros(len(train_df))
predictions = np.zeros(len(test_df))
feature_importance_df = pd.DataFrame()

for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_df.values, target.values)):
    print("Fold {}".format(fold_))
    trn_data = lgb.Dataset(train_df.iloc[trn_idx][features], label=target.iloc[trn_idx])
    val_data = lgb.Dataset(train_df.iloc[val_idx][features], label=target.iloc[val_idx])

    num_round = 1000000
    clf = lgb.train(param, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=1000, early_stopping_rounds = 3000)
    oof[val_idx] = clf.predict(train_df.iloc[val_idx][features], num_iteration=clf.best_iteration)
    
    fold_importance_df = pd.DataFrame()
    fold_importance_df["Feature"] = features
    fold_importance_df["importance"] = clf.feature_importance()
    fold_importance_df["fold"] = fold_ + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    
    predictions += clf.predict(test_df[features], num_iteration=clf.best_iteration) / folds.n_splits

print("CV score: {:<8.5f}".format(roc_auc_score(target, oof)))

Comprobemos la importancia de la característica.

In [ ]:
cols = (feature_importance_df[["Feature", "importance"]]
        .groupby("Feature")
        .mean()
        .sort_values(by="importance", ascending=False)[:150].index)
best_features = feature_importance_df.loc[feature_importance_df.Feature.isin(cols)]

plt.figure(figsize=(14,28))
sns.barplot(x="importance", y="Feature", data=best_features.sort_values(by="importance",ascending=False))
plt.title('Features importance (averaged/folds)')
plt.tight_layout()
plt.savefig('FI.png')

# <a id='6'>Sumisión</a>  


Presentamos la solución.

In [ ]:
sub_df = pd.DataFrame({"ID_code":test_df["ID_code"].values})
sub_df["target"] = predictions
sub_df.to_csv("submission.csv", index=False)


In [ ]:
from IPython.display import HTML
import pandas as pd
import numpy as np
import base64

# function that takes in a dataframe and creates a text link to  
# download it (will only work for files < 2MB or so)
def create_download_link(df, title = "Download CSV file", filename = "submission.csv"):  
    csv = df.to_csv()
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

# create a random sample dataframe
df = pd.DataFrame(np.random.randn(50, 4), columns=list('ABCD'))

# create a link to download the dataframe
create_download_link(df)

# ↓ ↓ ↓  Yay, download link! ↓ ↓ ↓